# Simple RAG Application using Langchain

### Table of Contents

1. **Simple LLM App**
    * Load LLM
    * Create Prompt Teamplate
    * Merge Prompt Template & LLM to create Chain
2. **Build RAG App**
    * Load External Data
    * Generate Embeddings
    * Build Index in Vector Store (using Embeddings)
    * Create Retrieval Chain
    
    
### Installation

* **pip install langchain**

## 1. Simple LLM App

### 1.1 Load LLM

In [1]:
from langchain_community.llms import Ollama

llm = Ollama(model="mistral") # base_url = 'http://localhost:11434'

llm

Ollama(model='mistral')

In [2]:
response = llm.invoke("Do you know about Claude 3?")

print(response)

 I'm unable to provide an answer specifically about "Claude 3" without more context. There are several people named Claude in various fields, and "Claude 3" could refer to a number of things, such as:

1. A third album or work by an artist named Claude.
2. A character or figure named Claude from a book, movie, or game with that name.
3. A third version or update of a software or product named Claude.

If you could please provide more information about which "Claude 3" you're referring to, I would be happy to help you out with any questions or information you might have.


### 1.2 Create Prompt Template

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an Artificial Intelligence News Reporter."),
    ("user", "{query}")
])

### 1.3 Merge Prompt Template & LLM to Create LangChain Chain

In [4]:
llm_app = prompt | llm

llm_app

ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an Artificial Intelligence News Reporter.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))])
| Ollama(model='mistral')

In [5]:
response = llm_app.invoke({"query": "Do you know about Claude 3?"})

print(response)

 I'm sorry for any confusion, but as of now, there isn't a widely known entity or technology referred to as "Claude 3." I suggest checking if you meant to ask about a specific person, company, or technological advancement by that name. If it's related to artificial intelligence or machine learning, feel free to provide more context or details so I can help you with accurate and up-to-date information.


## 2. Build RAG App

### 2.1 Load External Data

* **pip install beautifulsoup4**

In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.anthropic.com/news/claude-3-family")

docs = loader.load()

In [7]:
urls = [
        "https://www.anthropic.com/news/releasing-claude-instant-1-2",
        "https://www.anthropic.com/news/claude-pro",
        "https://www.anthropic.com/news/claude-2",
        "https://www.anthropic.com/news/claude-2-1",
        "https://www.anthropic.com/news/claude-2-1-prompting",
        "https://www.anthropic.com/news/claude-3-family",
        "https://www.anthropic.com/claude"
       ] 

docs = []
for url in urls:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())
    
len(docs)

7

In [8]:
docs[0]

Document(page_content='Releasing Claude Instant 1.2 \\ AnthropicClaudeAPIResearchCompanyNewsCareersAnnouncementsReleasing Claude Instant 1.2Aug 9, 2023●1 min readBusinesses working with Claude can now access our latest version of Claude Instant, version 1.2, available through our API.\xa0Claude Instant is our faster, lower-priced yet still very capable model, which can handle a range of tasks including casual dialogue, text analysis, summarization, and document comprehension.Claude Instant 1.2 incorporates the strengths of our latest\xa0model Claude 2\xa0in real-world use cases and shows significant gains in key areas like math, coding, reasoning, and safety. It generates longer, more structured responses and follows formatting instructions better. Instant 1.2 also shows improvements in quote extraction, multilingual capabilities, and question answering.Claude Instant 1.2 outperforms Claude Instant 1.1 on math and coding, achieving 58.7% on the Codex evaluation compared to 52.8% in our

### 2.2 Generate Embeddings

In [9]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings_llm = OllamaEmbeddings(model="llama2") # base_url = 'http://localhost:11434'

In [10]:
embeddings = embeddings_llm.embed_query("How are you?")

embeddings[:5]

[-0.5044223070144653,
 -0.39013269543647766,
 2.335892915725708,
 0.2539888620376587,
 -0.27720341086387634]

In [11]:
type(embeddings), len(embeddings)

(list, 4096)

In [12]:
embeddings = embeddings_llm.embed_documents([
                                "Claude 3 is latest Conversational AI Model from Anthropic.",
                                "Gemini is latest Conversational AI Model from Google.",
                                "Llama-2 is latest Conversational AI Model from Meta.",
                                "Mixtral is latest Conversational AI Model from Mistral AI.",
                                "GPT-4 is latest Conversational AI Model from OpenAI."
                               ])

len(embeddings), type(embeddings), len(embeddings[0])

(5, list, 4096)

### 2.3 Build Index in Vector Store

* **pip install faiss-cpu**

In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()

documents = text_splitter.split_documents(docs)

len(documents)

12

In [17]:
! pip install faiss-cpu
from langchain_community.vectorstores import FAISS

vector_index = FAISS.from_documents(documents, embeddings_llm)

vector_index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 1.1 MB/s eta 0:00:0000:0100:010m


There are 3 functions to create index.

* **from_documents()**
* **from_embeddings()**
* **from_texts()**

In [18]:
retriever = vector_index.as_retriever()

relevant_docs = retriever.invoke({"input": "Do you know about Claude 3?"})

len(relevant_docs)

4

In [19]:
relevant_docs

[Document(page_content="analysis of charts & graphs, financials and market trends, forecastingDifferentiatorHigher intelligence than any other model available.*1M tokens available for specific use cases, please inquire. Claude 3 Sonnet strikes the ideal balance between intelligence and speed—particularly for enterprise workloads. It delivers strong performance at a lower cost compared to its peers, and is engineered for high endurance in large-scale AI deployments.Cost [Input $/million tokens | Output $/million tokens]$3 | $15Context window200KPotential usesData processing: RAG or search & retrieval over vast amounts of knowledgeSales: product recommendations, forecasting, targeted marketingTime-saving tasks: code generation, quality control, parse text from imagesDifferentiatorMore affordable than other models with similar intelligence; better for scale.Claude 3 Haiku is our fastest, most compact model for near-instant responsiveness. It answers simple queries and requests with unmatc

In [20]:
for doc in relevant_docs:
    print(f"Title : {doc.metadata['title']}, Source: {doc.metadata['source']}")

Title : Introducing the next generation of Claude \ Anthropic, Source: https://www.anthropic.com/news/claude-3-family
Title : Releasing Claude Instant 1.2 \ Anthropic, Source: https://www.anthropic.com/news/releasing-claude-instant-1-2
Title : Long context prompting for Claude 2.1 \ Anthropic, Source: https://www.anthropic.com/news/claude-2-1-prompting
Title : Introducing Claude 2.1 \ Anthropic, Source: https://www.anthropic.com/news/claude-2-1


### 2.4 Create Retrieval Chain

In [21]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document

prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context and your internal knowledge.
Give priority to context and if you are not sure then say you are not aware of topic:

<context>
{context}
</context>

Question: {input}
""")

#document_chain  = prompt | llm 
document_chain = create_stuff_documents_chain(llm, prompt)

response = document_chain.invoke({
    "input": "Do you know about Claude 3?",
    "context": [Document(page_content="Claude 3 is latest Conversational AI Model from Anthropic.")]
})

print(response)

 Based on the provided context, yes, I do have knowledge that Claude 3 is the latest conversational AI model developed by Anthropic.


In [22]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fd0fd8d4ca0>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\nAnswer the following question based on the provided context and your internal knowledge.\nGive priority to context and if you are not sure then say you are not aware of topic:\n\n<context>\n{context}\n</context>\n\nQuestion: {input}\n'))])
            | Ollama(model='mistral')
            |

In [23]:
response = retrieval_chain.invoke({"input": "Do you know about Claude 3?"})

type(response)

dict

In [24]:
response.keys()

dict_keys(['input', 'context', 'answer'])